In [1]:
import io

poems = list(line[1:-1] for line in 
             io.open('../datasets/Shahnameh-Poems.txt', mode="r", encoding="utf-8").readlines())

import pandas as pd
import numpy as np
import hazm

poems = np.array(poems)
poems = np.apply_along_axis(' '.join, 1, poems.reshape(-1, 2))

df = pd.DataFrame(poems, columns=['poems'])

normalizer = hazm.Normalizer(token_based=True)

df['normal'] = df.poems.apply(normalizer.normalize)

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('count', CountVectorizer(analyzer='word', ngram_range=(1, 1), max_features=40_000)),
                 ('tfidf', TfidfTransformer(sublinear_tf=True))]).fit(df.normal)

tfidf_vec = pipe.transform(df.normal).toarray()
count_vec = pipe['count'].transform(df.normal).toarray()

In [3]:
def get_similars_by_cosine_distance(poem, n=10):
    vector = pipe.transform([poem]).toarray().squeeze()
    dist = 1 - tfidf_vec.dot(vector) / (np.linalg.norm(tfidf_vec) * np.linalg.norm(vector) + 1)    
    indx = dist.argsort()[:n]
    return list(zip(df['poems'][indx], dist[indx]))

In [4]:
get_similars_by_cosine_distance('چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن')

[('چواین نامور نامه آمد ببن ز من روی کشور شود پرسخن', 0.9955303422997667),
 ('چواین نامور نامه آید ببن زمن روی کشور شود پر سخن', 0.9968623324531146),
 ('چو آن نامهٔ قیصر آمد ببن جهاندار بشنید چندان سخن', 0.9984948139349977),
 ('چواین نامه آرند نزدیک تو پراندیشه کن رای تاریک ت', 0.9987269136180931),
 ('چواین نامه آمد بنزد گراز پر اندیشه شد کهتر دیوسا', 0.9987339556254979),
 ('چواین بشنود شاه خندان شود گشاده رخ و سیم دندان ش', 0.9988287774270274),
 ('سخنهای دستان چو آمد ببن یلان برگشادند یکسر سخن', 0.998914103926993),
 ('چواین نامه آرند نزد شما که فرخنده باد او رمزد شم', 0.9989375828660748),
 ('بیامد یکی پرسخن کفشگر چنین گفت کای شاه بیدادگر', 0.9989458990190347),
 ('سپهدار کابل بیامد ز شهر زبان پرسخن دل پر از کین ', 0.9989461654483616)]